In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from boruta import BorutaPy
from joblib import dump, load
bitter_features = pd.read_csv('test.csv')

replaced_data_mean = bitter_features.iloc[:, 10:]

y = bitter_features['Sweet']

# Features
X = replaced_data_mean

In [2]:
X

,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,"torsions_((C,3,1),(C,3,1),(N,3,1),(N,3,0))","torsions_((C,4,0),(N,3,0),(O,2,0),(C,3,1))","torsions_((C,4,0),(C,4,0),(N,3,0),(O,2,0))","torsions_((C,4,0),(C,4,0),(C,4,0),(F,1,0))","torsions_((C,3,1),(N,3,0),(S,4,0),(O,2,0))","torsions_((C,3,1),(N,3,0),(N,3,1),(C,3,1))","torsions_((C,3,1),(O,2,0),(C,3,1),(O,2,0))","torsions_((C,4,0),(O,2,0),(C,3,1),(O,2,0))","torsions_((C,4,0),(C,3,1),(O,2,0),(C,3,1))","torsions_((C,4,0),(N,3,0),(S,4,0),(O,2,0))"
0,1.150349,-1.644854,1.318011,1.396713,1.396713,1.485165,-1.862732,-1.150349,-0.551560,0.654950,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338
1,0.971599,-0.326891,0.971599,1.135299,0.808592,1.135299,-1.509191,0.984235,-2.153875,1.743479,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338
2,0.971599,-0.326891,0.971599,1.135299,0.808592,1.135299,-1.509191,0.984235,-2.153875,1.743479,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338
3,-1.959964,-1.091620,-2.080278,-1.281552,-1.959964,-1.213340,-2.497705,0.674490,-5.199338,2.241403,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338
4,1.959964,-0.841621,1.959964,1.959964,1.959964,1.959964,-0.560703,-0.047017,-0.551560,0.229153,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,2.497705,-1.485165,2.497705,2.497705,2.241403,2.241403,-0.819502,-0.645273,-0.506508,0.133561,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338
157,0.551560,0.253347,0.402250,0.294010,0.368501,0.213099,-0.031338,0.141468,-0.007833,-0.062707,...,2.080278,2.080278,2.080278,2.080278,1.862732,2.080278,2.080278,2.080278,2.080278,2.080278
158,-1.263944,1.743479,-1.336916,-1.439531,-1.336916,-1.509191,1.909094,1.356312,1.780464,-1.120502,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338
159,1.780464,-1.780464,1.862732,1.862732,1.862732,1.862732,-1.587056,-1.376231,-0.471197,0.070557,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338


In [3]:
import pickle
file = open('features/sweet/sweet_boruta2.p','rb')

boruta = pickle.load(file)

In [4]:
len(boruta)

458

In [5]:
X_transformed = X.reindex(columns=boruta).values

In [6]:
len(X_transformed)

161

In [7]:
file = open('final_model/adaboost.p','rb')
import pickle
best_model =pickle.load(file)

y_pred = best_model.predict(X_transformed)
y_pred_proba = best_model.predict_proba(X_transformed)[:, 1]
print("Bitter : ")
print(y_pred)
print(y_pred_proba)


Bitter : 
[False  True  True  True  True  True  True  True  True False False False
  True False False False  True False  True  True False False False False
 False  True  True False False False False False False False False False
 False False False False False False False  True False False False  True
 False  True False  True False False False  True False  True  True False
 False  True  True False False False False False False False  True  True
  True  True  True  True  True  True False False False False False False
  True False False False False False False  True False  True False  True
  True False False False False False False False False  True False False
  True  True  True False False False  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False False  True False  True
  True False False  True False]
[0.400537

In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, roc_auc_score,
    precision_score, recall_score, f1_score, average_precision_score
)
import numpy as np

# Model evaluation
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)
roc_auc = roc_auc_score(y, y_pred_proba)
aupr = average_precision_score(y, y_pred_proba)  # AUPR
sensitivity = recall  # Sensitivity (Sn) is the same as recall

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

# Calculate NER and Specificity
ner = fp / (tn + fp) if (tn + fp) > 0 else 0  # Avoid division by zero
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # Avoid division by zero

# Print results
print("Accuracy:", accuracy)
print("Precision (P):", precision)
print("Recall (R):", recall)
print("Sensitivity (Sn):", sensitivity)
print("F1-score (F1):", f1)
print("ROC-AUC Score (AuROC):", roc_auc)
print("AUPR Score (AuPR):", aupr)
print("Non Error Rate (NER):", 1-ner)
print("Specificity (SP):", specificity)
print("Classification Report:\n", classification_report(y, y_pred))

Accuracy: 0.7018633540372671
Precision (P): 0.875
Recall (R): 0.6481481481481481
Sensitivity (Sn): 0.6481481481481481
F1-score (F1): 0.7446808510638299
ROC-AUC Score (AuROC): 0.782669461914745
AUPR Score (AuPR): 0.8518690840204178
Non Error Rate (NER): 0.8113207547169812
Specificity (SP): 0.8113207547169812
Classification Report:
               precision    recall  f1-score   support

       False       0.53      0.81      0.64        53
        True       0.88      0.65      0.74       108

    accuracy                           0.70       161
   macro avg       0.70      0.73      0.69       161
weighted avg       0.76      0.70      0.71       161

